In [25]:
# HFT Research DMM Order Splitting
# Code by Michael Lee
# Objective of code: find trades that occurred at the same time at different exchanges
# Updated with Rolling Joint (2023.08.02)

In [26]:
import pandas as pd
import numpy as np
import gzip
import shutil
import warnings
warnings.filterwarnings("ignore")
import datetime

In [27]:
file_gz_stamp = 'SPIDER_2020-12-14.csv.gz' 

file_stamp = file_gz_stamp[:-3]
fin = file_gz_stamp
with gzip.open(fin, 'rb') as f:
        with open(file_stamp, 'wb') as f_out:
                shutil.copyfileobj(f, f_out)

In [28]:
df = pd.read_csv(file_stamp)
df

,okey_ts,okey_yr,okey_mn,okey_dy,okey_xx,okey_cp,timestamp,prtNumber,tradingSession,ticker_at,...,pnlM1,oBidM10,oAskM10,uBidM10,uAskM10,uPrcM10,sVolM10,sOpxM10,sDivM10,pnlM10
0,NMS,2021,3,17,35.0,Call,2020-12-14T08:00:00.003626Z,1406,PreMkt,EQT,...,0.0202,2.45,2.75,0.000000,0.000000,26.200121,1.0388,2.7702,0.0053,0.0202
1,NMS,2021,3,17,35.0,Call,2020-12-14T08:00:00.003637Z,1407,PreMkt,EQT,...,0.0202,2.45,2.75,0.000000,0.000000,26.200121,1.0388,2.7702,0.0053,0.0202
2,NMS,2021,3,19,100.0,Call,2020-12-14T08:05:15.349960Z,1553,PreMkt,EQT,...,-96.8900,3.06,3.16,99.989998,100.010002,100.000000,0.0000,0.1371,0.0000,-96.8900
3,NMS,2021,3,19,100.0,Call,2020-12-14T08:05:15.472337Z,1554,PreMkt,EQT,...,-96.8900,3.06,3.16,99.989998,100.010002,100.000000,0.0000,0.1371,0.0000,-96.8900
4,NMS,2021,3,19,100.0,Call,2020-12-14T08:05:15.473207Z,1555,PreMkt,EQT,...,-96.8900,3.06,3.16,99.989998,100.010002,100.000000,0.0000,0.1371,0.0000,-96.8900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5286164,NMS,2021,5,21,170.0,Call,2020-12-14T21:24:50.612049Z,9402369,PreMkt,EQT,...,-0.6912,4.40,6.40,156.399994,156.750000,156.399994,0.2529,5.4459,-0.0014,-0.6911
5286165,NMS,2020,12,16,375.0,Call,2020-12-14T21:26:53.790550Z,9402379,PreMkt,EQT,...,-0.0044,0.07,0.08,365.019989,365.089996,365.054993,0.1771,0.0766,0.0033,-0.0048
5286166,NMS,2020,12,16,357.0,Call,2020-12-14T21:27:02.915060Z,9402381,PreMkt,EQT,...,-0.9772,8.65,8.74,365.019989,365.089996,365.054993,0.2657,8.7285,0.0033,-0.9758
5286167,NMS,2020,12,18,375.0,Call,2020-12-14T21:41:06.564655Z,9402532,PreMkt,EQT,...,-0.0265,0.22,0.23,364.959992,365.000000,364.979981,0.1683,0.2221,-0.0054,-0.0267


In [29]:
df = df[df['ticker_tk']=='AAPL']
# Filters the file to only include Apple trades
df

,okey_ts,okey_yr,okey_mn,okey_dy,okey_xx,okey_cp,timestamp,prtNumber,tradingSession,ticker_at,...,pnlM1,oBidM10,oAskM10,uBidM10,uAskM10,uPrcM10,sVolM10,sOpxM10,sDivM10,pnlM10
6393,NMS,2020,12,24,110.0,Call,2020-12-14T14:30:00.004957Z,17150,RegularMkt,EQT,...,0.0785,13.20,13.30,123.070000,123.080002,123.074997,0.3923,13.2862,-0.0020,0.0428
7270,NMS,2020,12,18,121.0,Put,2020-12-14T14:30:00.533399Z,18029,RegularMkt,EQT,...,0.0592,0.93,0.94,123.070000,123.080002,123.074997,0.2836,0.9377,0.0031,-0.0175
7271,NMS,2020,12,18,122.0,Put,2020-12-14T14:30:00.533634Z,18030,RegularMkt,EQT,...,0.1512,1.26,1.27,123.070000,123.080002,123.074997,0.2777,1.2712,0.0031,0.0702
7272,NMS,2020,12,18,126.0,Call,2020-12-14T14:30:00.533865Z,18031,RegularMkt,EQT,...,-0.0614,0.66,0.68,123.070000,123.080002,123.074997,0.2725,0.6673,0.0031,-0.1231
7277,NMS,2020,12,24,130.0,Call,2020-12-14T14:30:00.534740Z,18036,RegularMkt,EQT,...,-0.0673,0.62,0.63,123.070000,123.080002,123.074997,0.3155,0.6232,-0.0020,-0.0972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5271203,NMS,2020,12,18,107.5,Put,2020-12-14T20:59:59.722299Z,9385982,RegularMkt,EQT,...,0.0013,0.04,0.08,121.849998,121.860001,121.854996,0.5268,0.0595,0.0000,0.0029
5271211,NMS,2020,12,31,105.0,Put,2020-12-14T20:59:59.735344Z,9385990,RegularMkt,EQT,...,0.0189,0.21,0.23,121.849998,121.860001,121.854996,0.4410,0.2249,-0.0094,0.0221
5271212,NMS,2020,12,31,105.0,Put,2020-12-14T20:59:59.735478Z,9385991,RegularMkt,EQT,...,0.0189,0.21,0.23,121.849998,121.860001,121.854996,0.4410,0.2249,-0.0094,0.0221
5271225,NMS,2020,12,31,121.0,Call,2020-12-14T20:59:59.751058Z,9386004,RegularMkt,EQT,...,0.1329,3.50,3.70,121.849998,121.860001,121.854996,0.3172,3.6880,-0.0094,0.1352


In [30]:
df['option_tk'] \
= df['okey_yr'].apply(lambda x: str(x)) \
+ df['okey_mn'].apply(lambda x: str(x)) \
+ df['okey_dy'].apply(lambda x: str(x)) \
+ (df['okey_xx']*10).apply(lambda x: str(int(x))) \
+ df['okey_cp'].apply(lambda x: str(x)[:1]) 

df

,okey_ts,okey_yr,okey_mn,okey_dy,okey_xx,okey_cp,timestamp,prtNumber,tradingSession,ticker_at,...,oBidM10,oAskM10,uBidM10,uAskM10,uPrcM10,sVolM10,sOpxM10,sDivM10,pnlM10,option_tk
6393,NMS,2020,12,24,110.0,Call,2020-12-14T14:30:00.004957Z,17150,RegularMkt,EQT,...,13.20,13.30,123.070000,123.080002,123.074997,0.3923,13.2862,-0.0020,0.0428,202012241100C
7270,NMS,2020,12,18,121.0,Put,2020-12-14T14:30:00.533399Z,18029,RegularMkt,EQT,...,0.93,0.94,123.070000,123.080002,123.074997,0.2836,0.9377,0.0031,-0.0175,202012181210P
7271,NMS,2020,12,18,122.0,Put,2020-12-14T14:30:00.533634Z,18030,RegularMkt,EQT,...,1.26,1.27,123.070000,123.080002,123.074997,0.2777,1.2712,0.0031,0.0702,202012181220P
7272,NMS,2020,12,18,126.0,Call,2020-12-14T14:30:00.533865Z,18031,RegularMkt,EQT,...,0.66,0.68,123.070000,123.080002,123.074997,0.2725,0.6673,0.0031,-0.1231,202012181260C
7277,NMS,2020,12,24,130.0,Call,2020-12-14T14:30:00.534740Z,18036,RegularMkt,EQT,...,0.62,0.63,123.070000,123.080002,123.074997,0.3155,0.6232,-0.0020,-0.0972,202012241300C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5271203,NMS,2020,12,18,107.5,Put,2020-12-14T20:59:59.722299Z,9385982,RegularMkt,EQT,...,0.04,0.08,121.849998,121.860001,121.854996,0.5268,0.0595,0.0000,0.0029,202012181075P
5271211,NMS,2020,12,31,105.0,Put,2020-12-14T20:59:59.735344Z,9385990,RegularMkt,EQT,...,0.21,0.23,121.849998,121.860001,121.854996,0.4410,0.2249,-0.0094,0.0221,202012311050P
5271212,NMS,2020,12,31,105.0,Put,2020-12-14T20:59:59.735478Z,9385991,RegularMkt,EQT,...,0.21,0.23,121.849998,121.860001,121.854996,0.4410,0.2249,-0.0094,0.0221,202012311050P
5271225,NMS,2020,12,31,121.0,Call,2020-12-14T20:59:59.751058Z,9386004,RegularMkt,EQT,...,3.50,3.70,121.849998,121.860001,121.854996,0.3172,3.6880,-0.0094,0.1352,202012311210C


In [31]:
df_CBOE = df[df['prtExch']=='CBOE']
# Create a new dataframe that only includes trades conducted at CBOE (Chicago Board Options Exchange)
df_CBOE

,okey_ts,okey_yr,okey_mn,okey_dy,okey_xx,okey_cp,timestamp,prtNumber,tradingSession,ticker_at,...,oBidM10,oAskM10,uBidM10,uAskM10,uPrcM10,sVolM10,sOpxM10,sDivM10,pnlM10,option_tk
17748,NMS,2020,12,18,125.0,Call,2020-12-14T14:30:02.634215Z,28542,RegularMkt,EQT,...,0.94,0.95,123.070000,123.080002,123.074997,0.2715,0.9466,0.0032,0.0032,202012181250C
18226,NMS,2020,12,31,124.0,Call,2020-12-14T14:30:02.704017Z,29021,RegularMkt,EQT,...,2.97,3.00,123.070000,123.080002,123.074997,0.3212,2.9891,-0.0080,-0.0593,202012311240C
18227,NMS,2020,12,31,124.0,Call,2020-12-14T14:30:02.704027Z,29022,RegularMkt,EQT,...,2.97,3.00,123.070000,123.080002,123.074997,0.3212,2.9891,-0.0080,-0.0593,202012311240C
18236,NMS,2020,12,31,131.0,Call,2020-12-14T14:30:02.705267Z,29031,RegularMkt,EQT,...,0.95,0.97,123.070000,123.080002,123.074997,0.3318,0.9625,-0.0080,-0.0160,202012311310C
18291,NMS,2020,12,18,121.0,Call,2020-12-14T14:30:02.711210Z,29086,RegularMkt,EQT,...,3.00,3.05,123.070000,123.080002,123.074997,0.2830,3.0147,0.0031,-0.0425,202012181210C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5270057,NMS,2020,12,24,119.0,Put,2020-12-14T20:59:57.913487Z,9384833,RegularMkt,EQT,...,1.30,1.37,121.849998,121.860001,121.854996,0.3057,1.3159,-0.0016,0.0256,202012241190P
5270062,NMS,2021,2,19,125.0,Call,2020-12-14T20:59:57.917600Z,9384838,RegularMkt,EQT,...,6.20,6.35,121.849998,121.860001,121.854996,0.3759,6.3273,-0.0036,-0.0135,20212191250C
5270063,NMS,2021,2,19,135.0,Call,2020-12-14T20:59:57.917600Z,9384839,RegularMkt,EQT,...,3.15,3.25,121.849998,121.860001,121.854996,0.3800,3.2393,-0.0036,-0.0164,20212191350C
5270188,NMS,2020,12,24,111.0,Call,2020-12-14T20:59:58.084236Z,9384964,RegularMkt,EQT,...,10.60,11.10,121.849998,121.860001,121.854996,0.3815,11.1319,-0.0016,-0.0250,202012241110C


In [32]:
df_CBOE['time_CBOE'] \
= pd.to_datetime(df_CBOE['timestamp']).apply(lambda x: x.timestamp())
# Add a new column that has timestamp for each trade in the CBOE dataframe

df_CBOE

,okey_ts,okey_yr,okey_mn,okey_dy,okey_xx,okey_cp,timestamp,prtNumber,tradingSession,ticker_at,...,oAskM10,uBidM10,uAskM10,uPrcM10,sVolM10,sOpxM10,sDivM10,pnlM10,option_tk,time_CBOE
17748,NMS,2020,12,18,125.0,Call,2020-12-14T14:30:02.634215Z,28542,RegularMkt,EQT,...,0.95,123.070000,123.080002,123.074997,0.2715,0.9466,0.0032,0.0032,202012181250C,1.607956e+09
18226,NMS,2020,12,31,124.0,Call,2020-12-14T14:30:02.704017Z,29021,RegularMkt,EQT,...,3.00,123.070000,123.080002,123.074997,0.3212,2.9891,-0.0080,-0.0593,202012311240C,1.607956e+09
18227,NMS,2020,12,31,124.0,Call,2020-12-14T14:30:02.704027Z,29022,RegularMkt,EQT,...,3.00,123.070000,123.080002,123.074997,0.3212,2.9891,-0.0080,-0.0593,202012311240C,1.607956e+09
18236,NMS,2020,12,31,131.0,Call,2020-12-14T14:30:02.705267Z,29031,RegularMkt,EQT,...,0.97,123.070000,123.080002,123.074997,0.3318,0.9625,-0.0080,-0.0160,202012311310C,1.607956e+09
18291,NMS,2020,12,18,121.0,Call,2020-12-14T14:30:02.711210Z,29086,RegularMkt,EQT,...,3.05,123.070000,123.080002,123.074997,0.2830,3.0147,0.0031,-0.0425,202012181210C,1.607956e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5270057,NMS,2020,12,24,119.0,Put,2020-12-14T20:59:57.913487Z,9384833,RegularMkt,EQT,...,1.37,121.849998,121.860001,121.854996,0.3057,1.3159,-0.0016,0.0256,202012241190P,1.607980e+09
5270062,NMS,2021,2,19,125.0,Call,2020-12-14T20:59:57.917600Z,9384838,RegularMkt,EQT,...,6.35,121.849998,121.860001,121.854996,0.3759,6.3273,-0.0036,-0.0135,20212191250C,1.607980e+09
5270063,NMS,2021,2,19,135.0,Call,2020-12-14T20:59:57.917600Z,9384839,RegularMkt,EQT,...,3.25,121.849998,121.860001,121.854996,0.3800,3.2393,-0.0036,-0.0164,20212191350C,1.607980e+09
5270188,NMS,2020,12,24,111.0,Call,2020-12-14T20:59:58.084236Z,9384964,RegularMkt,EQT,...,11.10,121.849998,121.860001,121.854996,0.3815,11.1319,-0.0016,-0.0250,202012241110C,1.607980e+09


In [33]:
df_xCBOE = df[df['prtExch']!='CBOE']
# Create a dataframe of all non-CBOE trades
df_xCBOE

,okey_ts,okey_yr,okey_mn,okey_dy,okey_xx,okey_cp,timestamp,prtNumber,tradingSession,ticker_at,...,oBidM10,oAskM10,uBidM10,uAskM10,uPrcM10,sVolM10,sOpxM10,sDivM10,pnlM10,option_tk
6393,NMS,2020,12,24,110.0,Call,2020-12-14T14:30:00.004957Z,17150,RegularMkt,EQT,...,13.20,13.30,123.070000,123.080002,123.074997,0.3923,13.2862,-0.0020,0.0428,202012241100C
7270,NMS,2020,12,18,121.0,Put,2020-12-14T14:30:00.533399Z,18029,RegularMkt,EQT,...,0.93,0.94,123.070000,123.080002,123.074997,0.2836,0.9377,0.0031,-0.0175,202012181210P
7271,NMS,2020,12,18,122.0,Put,2020-12-14T14:30:00.533634Z,18030,RegularMkt,EQT,...,1.26,1.27,123.070000,123.080002,123.074997,0.2777,1.2712,0.0031,0.0702,202012181220P
7272,NMS,2020,12,18,126.0,Call,2020-12-14T14:30:00.533865Z,18031,RegularMkt,EQT,...,0.66,0.68,123.070000,123.080002,123.074997,0.2725,0.6673,0.0031,-0.1231,202012181260C
7277,NMS,2020,12,24,130.0,Call,2020-12-14T14:30:00.534740Z,18036,RegularMkt,EQT,...,0.62,0.63,123.070000,123.080002,123.074997,0.3155,0.6232,-0.0020,-0.0972,202012241300C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5271203,NMS,2020,12,18,107.5,Put,2020-12-14T20:59:59.722299Z,9385982,RegularMkt,EQT,...,0.04,0.08,121.849998,121.860001,121.854996,0.5268,0.0595,0.0000,0.0029,202012181075P
5271211,NMS,2020,12,31,105.0,Put,2020-12-14T20:59:59.735344Z,9385990,RegularMkt,EQT,...,0.21,0.23,121.849998,121.860001,121.854996,0.4410,0.2249,-0.0094,0.0221,202012311050P
5271212,NMS,2020,12,31,105.0,Put,2020-12-14T20:59:59.735478Z,9385991,RegularMkt,EQT,...,0.21,0.23,121.849998,121.860001,121.854996,0.4410,0.2249,-0.0094,0.0221,202012311050P
5271225,NMS,2020,12,31,121.0,Call,2020-12-14T20:59:59.751058Z,9386004,RegularMkt,EQT,...,3.50,3.70,121.849998,121.860001,121.854996,0.3172,3.6880,-0.0094,0.1352,202012311210C


In [34]:
df_xCBOE['time_xCBOE'] \
= pd.to_datetime(df_xCBOE['timestamp']).apply(lambda x: x.timestamp())
# Create a new column for timestamp (non-CBOE) 

df_xCBOE

,okey_ts,okey_yr,okey_mn,okey_dy,okey_xx,okey_cp,timestamp,prtNumber,tradingSession,ticker_at,...,oAskM10,uBidM10,uAskM10,uPrcM10,sVolM10,sOpxM10,sDivM10,pnlM10,option_tk,time_xCBOE
6393,NMS,2020,12,24,110.0,Call,2020-12-14T14:30:00.004957Z,17150,RegularMkt,EQT,...,13.30,123.070000,123.080002,123.074997,0.3923,13.2862,-0.0020,0.0428,202012241100C,1.607956e+09
7270,NMS,2020,12,18,121.0,Put,2020-12-14T14:30:00.533399Z,18029,RegularMkt,EQT,...,0.94,123.070000,123.080002,123.074997,0.2836,0.9377,0.0031,-0.0175,202012181210P,1.607956e+09
7271,NMS,2020,12,18,122.0,Put,2020-12-14T14:30:00.533634Z,18030,RegularMkt,EQT,...,1.27,123.070000,123.080002,123.074997,0.2777,1.2712,0.0031,0.0702,202012181220P,1.607956e+09
7272,NMS,2020,12,18,126.0,Call,2020-12-14T14:30:00.533865Z,18031,RegularMkt,EQT,...,0.68,123.070000,123.080002,123.074997,0.2725,0.6673,0.0031,-0.1231,202012181260C,1.607956e+09
7277,NMS,2020,12,24,130.0,Call,2020-12-14T14:30:00.534740Z,18036,RegularMkt,EQT,...,0.63,123.070000,123.080002,123.074997,0.3155,0.6232,-0.0020,-0.0972,202012241300C,1.607956e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5271203,NMS,2020,12,18,107.5,Put,2020-12-14T20:59:59.722299Z,9385982,RegularMkt,EQT,...,0.08,121.849998,121.860001,121.854996,0.5268,0.0595,0.0000,0.0029,202012181075P,1.607980e+09
5271211,NMS,2020,12,31,105.0,Put,2020-12-14T20:59:59.735344Z,9385990,RegularMkt,EQT,...,0.23,121.849998,121.860001,121.854996,0.4410,0.2249,-0.0094,0.0221,202012311050P,1.607980e+09
5271212,NMS,2020,12,31,105.0,Put,2020-12-14T20:59:59.735478Z,9385991,RegularMkt,EQT,...,0.23,121.849998,121.860001,121.854996,0.4410,0.2249,-0.0094,0.0221,202012311050P,1.607980e+09
5271225,NMS,2020,12,31,121.0,Call,2020-12-14T20:59:59.751058Z,9386004,RegularMkt,EQT,...,3.70,121.849998,121.860001,121.854996,0.3172,3.6880,-0.0094,0.1352,202012311210C,1.607980e+09


In [35]:
specialist_list = pd.read_csv('Specialist_Long.csv')
specialist_list = specialist_list[specialist_list['SYM_ROOT']=='AAPL']
#specialist_list = specialist_list[specialist_list['prtExch']!='CBOE']
specialist_list['Specialist'].unique()
# Specialist list for reference

specialist_list

,SYM_ROOT,prtExch,Specialist
7,AAPL,CBOE,SIG
4090,AAPL,EDGX,SIG
7778,AAPL,MIAX,Morgan Stanley
11242,AAPL,EMLD,IMC
14708,AAPL,ISE,Citadel
18104,AAPL,GMNI,Two Sigma
21126,AAPL,MRCY,SIG
24191,AAPL,PHLX,Citadel
28033,AAPL,NQBX,Citadel
29760,AAPL,NYSE,SIG


In [36]:
CBOESL = specialist_list[specialist_list['prtExch']=='CBOE'] 
CBOE_specialist = specialist_list['Specialist'].iloc[0]
#print(CBOE_specialist)

Exchange_List = []
specialist_CBOEdf = specialist_list[specialist_list['Specialist'] == CBOE_specialist]
specialist_CBOEdf = specialist_CBOEdf[specialist_CBOEdf['prtExch'] != 'CBOE']
for k in range(len(specialist_CBOEdf)):
    Exchange_List.append(specialist_CBOEdf['prtExch'].iloc[k])

print(Exchange_List)


['EDGX', 'MRCY', 'NYSE']


In [37]:
# Note: the following code block is incomplete (will need more time to make the result data more legible)

# Final df columns: CBOE, option_tk, time at CBOE, time at non-CBOE, time diff, corresponding DMM (non-CBOE)
list_closetime = []

for i in range(len(df_CBOE['option_tk'])):
    # Loops through the CBOE list, but just for each unique option
    CBOE_tk = df_CBOE['option_tk'].iloc[i]
    CBOE_time = df_CBOE['time_CBOE'].iloc[i]
    # Define variables current option_tk + trading time (for reference)

    df_optiontk = df_xCBOE[df_xCBOE['option_tk'] == CBOE_tk]
    # Filter the non-CBOE dataframe so that it only includes the option that this iteration of the loop correspond to
    # For example: if this iteration of the outer loop is dealing with option_tk 202012181250C,
    # We will only look at entries in the Non-CBOE dataframe with option_tk value of 202012181250C.

    # Define the empty dictionary
    cena = []
    td_dict = {}
    for j in range(len(df_optiontk)):
        symb = df['ticker_tk'].iloc[0]
        optk = CBOE_tk
        timeCBOE = CBOE_time
        timeNotCBOE = df_optiontk['time_xCBOE'].iloc[j]
        timediff = np.abs(timeCBOE - timeNotCBOE)
        exchange = df_optiontk['prtExch'].iloc[j]
        # DMM = specialist_list[specialist_list['prtExch']==exchange]

        td_dict[timediff] = (symb, optk, timeCBOE, timeNotCBOE, timediff, exchange) 
        # Dictionary with key = time difference, value = tuple with everything
        # Create a dictionary of tuples, find closest match

    # Find smallest key in the dictionary
    
    smallest_td = min(td_dict, default=None)
    if smallest_td != None:
        cena.append(td_dict[smallest_td][0])
        cena.append(td_dict[smallest_td][1])
        cena.append(td_dict[smallest_td][2])
        cena.append(td_dict[smallest_td][3])
        cena.append(td_dict[smallest_td][4])
        cena.append(td_dict[smallest_td][5])
    
    if smallest_td != None and td_dict[smallest_td][4] <= 5:
        list_closetime.append(cena)
    

df_result = pd.DataFrame(list_closetime, columns=['Symbol', 'Option_tk', 'Time at CBOE', 'Time at NonCBOE', 'Time Difference', 'prtExch'])

df_result
# The explanation from results from the code above is listed in the 201214-AAPL-AAPL file that's included besides this file.

,Symbol,Option_tk,Time at CBOE,Time at NonCBOE,Time Difference,prtExch
0,AAPL,202012181250C,1.607956e+09,1.607956e+09,0.211688,PHLX
1,AAPL,202012311240C,1.607956e+09,1.607956e+09,0.036679,GMNI
2,AAPL,202012311240C,1.607956e+09,1.607956e+09,0.036669,GMNI
3,AAPL,202012311310C,1.607956e+09,1.607956e+09,0.760163,PHLX
4,AAPL,202012181210C,1.607956e+09,1.607956e+09,0.113131,NYSE
...,...,...,...,...,...,...
4924,AAPL,202012181260C,1.607980e+09,1.607980e+09,2.125806,C2
4925,AAPL,202012241250P,1.607980e+09,1.607980e+09,0.039453,PHLX
4926,AAPL,202012241190P,1.607980e+09,1.607980e+09,0.039463,PHLX
4927,AAPL,20212191250C,1.607980e+09,1.607980e+09,0.112206,BATS


In [38]:
#df_result = df_result[df_result['Time Difference'] <= 5]
#Filter by time

#df_result

In [39]:
# To Prof Ernst: This is the part I'm very unsure about

#print(Exchange_List)
df_res = df_result[df_result['prtExch']=='CBOE'] #Declare an empty dataframe

for s in range(len(Exchange_List)):
    
  df_temp = df_result[df_result['prtExch']==Exchange_List[s]]
  df_res = pd.concat([df_res, df_temp]) # <== I tried to merge the two dataframes at each iteration? but it doesnt seem to work
   

df_res
#finale = pd.DataFrame(f_list, columns=['Symbol', 'Option_tk', 'Time at CBOE', 'Time at NonCBOE', 'Time Difference', 'prtExch'])


,Symbol,Option_tk,Time at CBOE,Time at NonCBOE,Time Difference,prtExch
4,AAPL,202012181210C,1.607956e+09,1.607956e+09,0.113131,NYSE
9,AAPL,202012181220P,1.607956e+09,1.607956e+09,0.183939,NYSE
54,AAPL,202012181210C,1.607956e+09,1.607956e+09,0.267499,NYSE
99,AAPL,202012181220C,1.607956e+09,1.607956e+09,0.266495,NYSE
100,AAPL,202012181220C,1.607956e+09,1.607956e+09,0.266507,NYSE
...,...,...,...,...,...,...
4643,AAPL,20211151225C,1.607979e+09,1.607979e+09,0.051087,NYSE
4644,AAPL,20211151450C,1.607979e+09,1.607979e+09,0.051092,NYSE
4645,AAPL,20211151450C,1.607979e+09,1.607979e+09,0.051092,NYSE
4654,AAPL,20211151250C,1.607979e+09,1.607979e+09,1.393107,NYSE
